In [1]:
import pickle
import json
import numpy as np
import os

In [2]:
#data
path_layer1 ='/srv/datasets2/recipe1m+/utfood_3.10/layer1.json'
with open(path_layer1, "r") as f:
    layer1 = json.load(f)
with open('/srv/datasets2/recipe1m+/food.com_data_rescaled/nutr/food.com_nutr_g_per_recipe.json') as f:
    nutr_per_recipes = json.load(f)
with open('/srv/datasets2/recipe1m+/food.com_data_rescaled/nutr/simplified_food_ids_per_recipes.json') as f:
    food_ids_per_recipes = json.load(f)
with open('/srv/datasets2/recipe1m+/food.com_data_rescaled/nutr/simplified_weighted_food_ids_per_recipes.json') as f:
    weighted_food_ids_per_recipes = json.load(f)
with open('ingredient_meds.json') as f:
    ingredient_meds = json.load(f)
with open('../out/models/nutr_ingr_only/base_20ep/food_test/out_nutr.json') as f:
    pred_nutr = json.load(f)
with open('../out/models/nutr_ingr_only/base_20ep/food_test/out_ingrs.json') as f:
    pred_ingrs = json.load(f)

In [3]:
dir = 'deep_ht/direct_ingrs_3_branches/no_nutr/food_test_recipe_only'
im2recipe = np.load(f'../out/models/{dir}/im2recipe.npy')
recipe2im = np.load(f'../out/models/{dir}/recipe2im.npy')
with open(f'../out/models/{dir}/img_path','rb') as f:
    img_path = pickle.load(f)
with open(f'../out/models/{dir}/saved_ids','rb') as f:
    saved_ids = pickle.load(f)

In [129]:
im2recipe_dict = {}
for i, sorted_recipe in enumerate(im2recipe):
    avail_sorted_recipe = [saved_ids[idx] for idx in sorted_recipe][:100]
    im2recipe_dict[saved_ids[i]] = avail_sorted_recipe
save_path = f'results_100_ret/{dir}.json'
os.makedirs(os.path.dirname(save_path),exist_ok=True)
with open(save_path,'w') as f:
    json.dump(im2recipe_dict,f,indent=2)

In [130]:
recipe2im_dict = {}
for i, sorted_recipe in enumerate(recipe2im.T):
    avail_sorted_recipe = [saved_ids[idx] for idx in sorted_recipe][:100]
    recipe2im_dict[saved_ids[i]] = avail_sorted_recipe
save_path = f'results_100_ret_rec_to_im/{dir}.json'
os.makedirs(os.path.dirname(save_path),exist_ok=True)
with open(save_path,'w') as f:
    json.dump(recipe2im_dict,f,indent=2)

In [131]:
def cal_recall(im2recipe_dict, top_k):
    running_acc = 0
    for recipe_id, results in im2recipe_dict.items():
        top_results = results[:top_k]
        # print(top_results)
        if recipe_id in top_results:
            running_acc += 1
    acc = running_acc / len(im2recipe_dict)
    return acc


In [37]:
print(cal_recall(im2recipe_dict,1))
print(cal_recall(im2recipe_dict,5))
print(cal_recall(im2recipe_dict,10))

0.21275720164609052
0.5084362139917695
0.6551440329218107


In [38]:
print(cal_recall(recipe2im_dict,1))
print(cal_recall(recipe2im_dict,5))
print(cal_recall(recipe2im_dict,10))

0.1685185185185185
0.41316872427983536
0.558641975308642


In [4]:
def cal_iou(input_1, input_2):
    input_1 = set(input_1)
    input_2 = set(input_2)
    u = input_1.union(input_2)
    i = input_1.intersection(input_2)
    return len(i) / len(u)

def cal_weighted_iou(input_1, input_2, w_1, w_2):
    all_weight = sum(w_1) + sum(w_2)
    intersect_weight = 0.
    for i, ingr in enumerate(input_1):
        if ingr in input_2:
            j = input_2.index(ingr)
            intersect_weight += w_1[i] + w_2[j]
    return intersect_weight / (all_weight + 1e-8)

def cal_norm_weighted_iou(input_1, input_2, w_1, w_2):
    all_weight = sum(w_1) + sum(w_2)
    intersect_weight = 0.
    for i, ingr in enumerate(input_1):
        if ingr in input_2:
            j = input_2.index(ingr)
            intersect_weight += min(w_1[i],w_2[j])
            all_weight -= min(w_1[i],w_2[j])
    return intersect_weight / (all_weight + 1e-8)

def cal_mean_iou(im2recipe_dict,food_ids_per_recipes,weighted_food_ids_per_recipes):
    ious = []
    weighted_ious = []
    for recipe_id, results in im2recipe_dict.items():
        result = results[0]
        ious.append(cal_iou(food_ids_per_recipes[recipe_id],food_ids_per_recipes[result]))
        weighted_ious.append(cal_weighted_iou(food_ids_per_recipes[recipe_id],food_ids_per_recipes[result],weighted_food_ids_per_recipes[recipe_id]['weight'],weighted_food_ids_per_recipes[result]['weight']))
    iou = sum(ious)/len(ious) * 100
    weighted_iou = sum(weighted_ious)/len(weighted_ious) * 100
    return iou, weighted_iou

def cal_mase(pred, gt):
    # ms = {'energy': 211.98462376370378, 'fat': 10.765786025555727, 'carb': 22.87488284825082, 'protein': 6.497832050027167}
    mads = {'energy': 100.39411644302027, 'fat': 7.139428395140109, 'carb': 15.985982100326039, 'protein': 3.841522669109835}
    aes = {}
    ases = {}
    for nutr_name in gt.keys():
        aes[nutr_name] = abs(pred[nutr_name] - gt[nutr_name])
        ases[nutr_name] = aes[nutr_name] / mads[nutr_name]
    mase = sum(list(ases.values()))/len(list(gt.keys()))
    return aes, ases, mase

In [5]:
dir = 'deep_ht/direct_ingrs_3_branches/no_nutr/food_test_recipe_only'
with open(f'results_100_ret/{dir}.json') as f:
    im2recipe_dict = json.load(f)

In [6]:
im2recipe_dict_reranked = {}
k = 20
big_k = 20
iou_thresh = 0.3
for query, sorted_recipe in im2recipe_dict.items():
    pseudo_nutr = pred_nutr[query]
    pseudo_ingr = pred_ingrs[query]
    sorted_recipe = [recipe for recipe in sorted_recipe if recipe != query]
    reranked_recipe = sorted_recipe
    reranked_recipe = [recipe for recipe in sorted_recipe if cal_iou(pseudo_ingr,food_ids_per_recipes[recipe]) > iou_thresh]

    if len(reranked_recipe) < k:
        reranked_recipe = sorted(sorted_recipe,key = lambda x: cal_iou(food_ids_per_recipes[x],pseudo_ingr),reverse=True)[:big_k]
    reranked_recipe = sorted(reranked_recipe,key= lambda x: cal_mase(nutr_per_recipes[x],pseudo_nutr)[2])
    im2recipe_dict_reranked[query] = reranked_recipe
save_path = f'results/reranked_100_direct/{dir}.json'
os.makedirs(os.path.dirname(save_path),exist_ok=True)
with open(save_path,'w') as f:
    json.dump(im2recipe_dict_reranked,f,indent=2)

In [6]:
im2recipe_dict_reranked = {}
for query, sorted_recipe in im2recipe_dict.items():
    pseudo_nutr = pred_nutr[query]
    pseudo_ingr = pred_ingrs[query]
    reranked_recipe = [recipe for recipe in sorted_recipe if recipe != query]
    reranked_recipe = sorted(reranked_recipe,key= lambda x: cal_mase(nutr_per_recipes[x],pseudo_nutr)[2])
    im2recipe_dict_reranked[query] = reranked_recipe
save_path = f'results/reranked_100_wo_ingr_direct/{dir}.json'
os.makedirs(os.path.dirname(save_path),exist_ok=True)
with open(save_path,'w') as f:
    json.dump(im2recipe_dict_reranked,f,indent=2)

In [7]:
im2recipe_dict_reranked = {}
for query, sorted_recipe in im2recipe_dict.items():
    pseudo_nutr = pred_nutr[query]
    pseudo_ingr = pred_ingrs[query]
    sorted_recipe = [recipe for recipe in sorted_recipe if recipe != query]
    reranked_recipe = sorted(sorted_recipe,key = lambda x: cal_iou(food_ids_per_recipes[x],pseudo_ingr),reverse=True)
    im2recipe_dict_reranked[query] = reranked_recipe
save_path = f'results/reranked_100_wo_nutr_direct/{dir}.json'
os.makedirs(os.path.dirname(save_path),exist_ok=True)
with open(save_path,'w') as f:
    json.dump(im2recipe_dict_reranked,f,indent=2)